# Using the `register_name` kwarg to select how a parameter is saved to a dataset

This example notebook shows how to use the `register_name` keyword argument to control the name used to save a parameter to a dataset


## Setup

### Module Imports


In [7]:
from functools import partial
from pprint import pprint

import numpy as np

from qcodes.dataset import (
    do0d,
    do1d,
    initialise_or_create_database_at,
    load_or_create_experiment,
)
from qcodes.parameters import ManualParameter, Parameter, ParameterWithSetpoints
from qcodes.validators import Arrays

### Database and Experiment creation

In [2]:
initialise_or_create_database_at("./register_name_example.db")
load_or_create_experiment("register_name")

## Creating a Parameter with a `register_name`

It is simple to create a parameter with a `register_name`. Simply provide the keyword and the desired name to the parameter initialization. 

In [8]:
indep_param = ManualParameter(
    "indep_param", initial_value=1, register_name="renamed_indep_param"
)
dep_param = ManualParameter(
    "dep_param", initial_value=2, register_name="renamed_dep_param"
)

ds, _, _ = do1d(indep_param, 0, 1, 101, 0, dep_param)
pprint(ds.get_parameters())

Starting experimental run with id: 5. Using 'qcodes.dataset.do1d'
[ParamSpec('renamed_indep_param', 'numeric', 'indep_param', '', inferred_from=[], depends_on=[]),
 ParamSpec('renamed_dep_param', 'numeric', 'dep_param', '', inferred_from=[], depends_on=['renamed_indep_param'])]


Note that the dataset's ParamSpecs have replaced the usual parameter name with the register name.

Why is this useful?
---
Imagine that you have a parameter that is defined as part of a complex instrument. The parameter's full_name may be something like:

`instrument_submodule1_submodule2_param_full_name`

But you only really care about `param_full_name`. The `register_name` allows the user to define the string saved to the dataset decoupling the structure in the instrument.

## ParameterWithSetpoints

`register_name` also works with the ParameterWithSetpoints

In [9]:
setpoints_param = Parameter(
    name="setpoints_param",
    get_cmd=partial(np.linspace, 0, 1, 101),
    vals=Arrays(shape=(101,)),
    register_name="renamed_setpoints",
)
meas_param = ParameterWithSetpoints(
    "meas_param",
    setpoints=(setpoints_param,),
    get_cmd=partial(np.linspace, 0, -1, 101),
    vals=Arrays(
        shape=(101,),
        valid_types=[np.integer, np.floating, np.complexfloating],
    ),
    register_name="renamed_meas_param",
)

ds, _, _ = do0d(meas_param)
pprint(ds.get_parameters())

Starting experimental run with id: 6. Using 'qcodes.dataset.do0d'
[ParamSpec('renamed_setpoints', 'array', 'setpoints_param', '', inferred_from=[], depends_on=[]),
 ParamSpec('renamed_meas_param', 'array', 'meas_param', '', inferred_from=[], depends_on=['renamed_setpoints'])]


Note: `register_name` does not currently work with `MultiParameter` or `ArrayParameter`